# NLP - Word representation

## The imports

In [6]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, TextVectorization, Dense, Flatten, Embedding

## The corpus

In [23]:
texts = np.array(["I like chocolate",
            "I like tea",
            "You like chocolate",
            'You hate beer',
            'I hate wine'])
labels = np.array([1,1,1,0,0])

## BOW representation (the fist week)

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [35]:
# with Keras preprocessing layer
vectorize_layer = TextVectorization(output_mode='count', ngrams=(1,2))
# Fit the layer with the corpus
vectorize_layer.adapt(texts)
print(vectorize_layer(texts), '\n')
print(vectorize_layer.get_vocabulary())

tf.Tensor(
[[0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.]], shape=(5, 17), dtype=float32) 

['[UNK]', 'like', 'i', 'you', 'like chocolate', 'i like', 'hate', 'chocolate', 'you like', 'you hate', 'wine', 'tea', 'like tea', 'i hate', 'hate wine', 'hate beer', 'beer']


In [36]:
# define the model
input_ = Input(shape=(1,), dtype=tf.string)
x = vectorize_layer(input_)
hidden = Dense(32, activation='relu')(x)
output_ = Dense(1, activation='sigmoid')(hidden)
model = Model(input_, output_)

# summarize the model
model.summary()

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(texts, labels, epochs=10, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(texts, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_22 (Text  (None, 17)               0         
 Vectorization)                                                  
                                                                 
 dense_6 (Dense)             (None, 32)                576       
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________
Accuracy: 60.000002


## Keras word embedding

In [46]:
# Constants
vocab_size = 10  # Maximum vocab size.
max_len = 5      # Sequence length to pad the outputs to.
embedding_size = 8

# with Keras preprocessing layer
vectorize_layer = TextVectorization(max_tokens=vocab_size,
                                    output_mode='int',
                                    output_sequence_length=max_len)
# Fit the layer with the corpus
vectorize_layer.adapt(texts)
print(vectorize_layer(texts), '\n')
print(vectorize_layer.get_vocabulary())

tf.Tensor(
[[3 2 6 0 0]
 [3 2 8 0 0]
 [4 2 6 0 0]
 [4 5 9 0 0]
 [3 5 7 0 0]], shape=(5, 5), dtype=int64) 

['', '[UNK]', 'like', 'i', 'you', 'hate', 'chocolate', 'wine', 'tea', 'beer']


In [47]:
# define the model
input_ = Input(shape=(1,), dtype=tf.string)
x = vectorize_layer(input_)
x = Embedding(vocab_size, embedding_size, name="Embedding")(x)
x = Flatten()(x)
hidden = Dense(32, activation="relu")(x)
output_ = Dense(1, activation='sigmoid')(hidden)
model = Model(input_, output_)

# summarize the model
model.summary()

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(texts, labels, epochs=10, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(texts, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_29 (Text  (None, 5)                0         
 Vectorization)                                                  
                                                                 
 Embedding (Embedding)       (None, 5, 8)              80        
                                                                 
 flatten_2 (Flatten)         (None, 40)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                1312      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                           

Accuracy: 80.000001


## Use a pre-trained embedding : Glove/Word2Vec/FastText embedding

**Traditional word embedding** techniques (Glove/Word2Vec/FastText) learn a global word embedding. They first build a global vocabulary using unique words in the documents by ignoring the meaning of words in different context. Then, similar representations are learnt for the words appeared more frequently close each other in the documents. The problem is that in such word representations the words' contextual meaning (the meaning derived from the words' surroundings), is ignored. For example, only one representation is learnt for "left" in sentence "I left my phone on the left side of the table." However, "left" has two different meanings in the sentence, and needs to have two different representations in the embedding space.

For example, consider the two sentences:

1. I will show you a valid point of reference and talk to the point.
1. Where have you placed the point.

The word embeddings from a pre-trained embeddings such as word2vec, the embeddings for the word 'point' is same for both of its occurrences in example 1 and also the same for the word 'point' in example 2. (all three occurrences has same embeddings).

In [49]:
# Same steps as Keras Embedding
vocab_size = 10  # Maximum vocab size.
max_len = 5      # Sequence length to pad the outputs to.
hidden_size = 16

vectorizer = TextVectorization(max_tokens=vocab_size, output_sequence_length=max_len)
vectorizer.adapt(texts)
print(vectorizer(texts))

tf.Tensor(
[[3 2 6 0 0]
 [3 2 8 0 0]
 [4 2 6 0 0]
 [4 5 9 0 0]
 [3 5 7 0 0]], shape=(5, 5), dtype=int64)


In [50]:
# Build word dict
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
word_index

{'': 0,
 '[UNK]': 1,
 'like': 2,
 'i': 3,
 'you': 4,
 'hate': 5,
 'chocolate': 6,
 'wine': 7,
 'tea': 8,
 'beer': 9}

In [51]:
# Download the pre-trained embedding matrix for exemple from glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-02-07 13:38:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-02-07 13:38:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-02-07 13:38:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [52]:
# Make a dict mapping words (strings) to their NumPy vector representation:
path_to_glove_file = "/content/glove.6B.50d.txt"

# pre-trained embedding matrix
embedding_dim = 50 # fixed by thepre-trained embedding matrix

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


Let's prepare a corresponding embedding matrix that we can use in a Keras Embedding layer. It's a simple NumPy matrix where entry at index i is the pre-trained vector for the word of index i in our vectorizer's vocabulary.

In [54]:
num_tokens = len(voc) + 2 # UNK/OOV and PAD
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    print(word, i)
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

 0
[UNK] 1
like 2
i 3
you 4
hate 5
chocolate 6
wine 7
tea 8
beer 9
Converted 8 words (2 misses)


In [55]:
# Initialize the Embedding layer with the weight of each word
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)
num_tokens,embedding_dim, max_len

(12, 50, 5)

In [56]:
# define the model
input_ = Input(shape=(1,), dtype=tf.string)
x = vectorize_layer(input_)
x = embedding_layer(x)
x = Flatten()(x)
hidden = Dense(32, activation="relu")(x)
output_ = Dense(1, activation='sigmoid')(hidden)
model = Model(input_, output_)

# summarize the model
model.summary()

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(texts, labels, epochs=10, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(texts, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_29 (Text  (None, 5)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 5, 50)             600       
                                                                 
 flatten_3 (Flatten)         (None, 250)               0         
                                                                 
 dense_12 (Dense)            (None, 32)                8032      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                           

## Train your own Word2Vec model with gensim

In [57]:
#!pip install gensim

To begin with, you need data to train a model. We will use part of the Brown corpus.

In [59]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
from gensim.models import Word2Vec

train_set = brown.sents()[:10000]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


Let's go ahead and train a model on our corpus. Don't worry about the training parameters much for now, we'll revisit them later.

In [60]:
model = Word2Vec(sentences=train_set, size=embedding_dim, window=5, min_count=1, workers=4)
#model = Word2Vec(sentences=common_texts, vector_size=embedding_dim, window=5, min_count=1, workers=4)

Once we have our model, we can use it.

The main part of the model is model.wv\ , where "wv" stands for "word vectors".

In [61]:
vector = model.wv['university']  # get numpy vector of a word
vector

array([ 5.29496819e-02, -4.46372516e-02, -3.79595250e-01,  2.95849115e-01,
        1.08769089e-01, -7.19127432e-02, -4.45467532e-01, -2.35504746e-01,
       -1.18837059e-02,  1.12652726e-01, -1.52235314e-01, -3.19412276e-02,
        2.01125339e-01, -2.01649100e-01,  5.45171946e-02,  1.28907412e-01,
       -3.49997550e-01,  5.38622178e-02,  2.81889439e-01, -6.46037683e-02,
        7.88322538e-02, -4.77848798e-02, -1.81927830e-01,  1.71632349e-01,
       -3.46808578e-04,  8.17758739e-02, -9.10655409e-02,  8.46376643e-02,
       -1.98376417e-01, -8.74609277e-02,  3.65990520e-01, -2.16667965e-01,
        2.88263410e-01,  3.20855714e-02,  3.44788246e-02,  6.63531944e-02,
       -2.13096783e-01, -8.97528976e-02,  1.00009628e-01,  2.05221653e-01,
       -2.36675024e-01, -5.57820685e-02,  3.92683707e-02, -2.90945452e-02,
        1.91943586e-01,  1.19256839e-01,  1.04865886e-01,  1.22915819e-01,
       -1.84769899e-01,  1.04869008e-01], dtype=float32)

In [70]:
model.similarity('university','school')

0.998015

In [71]:
sims = model.wv.most_similar('university', topn=10)  # get other similar words
sims

[('meeting', 0.9993147253990173),
 ('program', 0.9993127584457397),
 ('death', 0.9992231130599976),
 ('state', 0.9992156624794006),
 ('attention', 0.9991841316223145),
 ('same', 0.9991660714149475),
 ('series', 0.9991559982299805),
 ('other', 0.9991316199302673),
 ('new', 0.9991243481636047),
 ('board', 0.9991194605827332)]

Training non-trivial models can take time.  Once the model is built, it can be saved using standard gensim methods:

In [72]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    print(temporary_filepath)
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = Word2Vec.load(temporary_filepath)

/tmp/gensim-model-km2g9gpn


If you save the model you can continue training it later:

In [74]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
new_model.train([word_tokenize(sent) for sent in texts], total_examples=1, epochs=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


(15, 15)

If you no longer need to retrain the model, it can be saved with only the vectors and their keys. This results in a much smaller and faster object that can be loaded more quickly.

In [75]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = new_model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
new_word_vectors = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = new_word_vectors['university']  # Get numpy vector of a word
vector

array([ 5.29496819e-02, -4.46372516e-02, -3.79595250e-01,  2.95849115e-01,
        1.08769089e-01, -7.19127432e-02, -4.45467532e-01, -2.35504746e-01,
       -1.18837059e-02,  1.12652726e-01, -1.52235314e-01, -3.19412276e-02,
        2.01125339e-01, -2.01649100e-01,  5.45171946e-02,  1.28907412e-01,
       -3.49997550e-01,  5.38622178e-02,  2.81889439e-01, -6.46037683e-02,
        7.88322538e-02, -4.77848798e-02, -1.81927830e-01,  1.71632349e-01,
       -3.46808578e-04,  8.17758739e-02, -9.10655409e-02,  8.46376643e-02,
       -1.98376417e-01, -8.74609277e-02,  3.65990520e-01, -2.16667965e-01,
        2.88263410e-01,  3.20855714e-02,  3.44788246e-02,  6.63531944e-02,
       -2.13096783e-01, -8.97528976e-02,  1.00009628e-01,  2.05221653e-01,
       -2.36675024e-01, -5.57820685e-02,  3.92683707e-02, -2.90945452e-02,
        1.91943586e-01,  1.19256839e-01,  1.04865886e-01,  1.22915819e-01,
       -1.84769899e-01,  1.04869008e-01], dtype=float32)

You can then use the template exactly as if it were a Glove/Word2Vec/FastText template retrieved from the Internet.

In [76]:
# Build word dict
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
word_index

{'': 0,
 '[UNK]': 1,
 'like': 2,
 'i': 3,
 'you': 4,
 'hate': 5,
 'chocolate': 6,
 'wine': 7,
 'tea': 8,
 'beer': 9}

In [77]:
num_tokens = len(voc) + 2
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    try:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = new_word_vectors[word]
        hits += 1
    except :
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 7 words (3 misses)


In [78]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
    name="Embedding"
)

In [79]:
# define the model
input_ = Input(shape=(1,), dtype=tf.string, name="Input")
x = vectorize_layer(input_)
x = embedding_layer(x)
x = Flatten()(x)
hidden = Dense(32, activation="relu", name="Hidden")(x)
output_ = Dense(1, activation='sigmoid', name="Output")(hidden)
model = Model(input_, output_)

# summarize the model
model.summary()

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(texts, labels, epochs=10, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(texts, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 1)]               0         
                                                                 
 text_vectorization_29 (Text  (None, 5)                0         
 Vectorization)                                                  
                                                                 
 Embedding (Embedding)       (None, 5, 50)             600       
                                                                 
 flatten_4 (Flatten)         (None, 250)               0         
                                                                 
 Hidden (Dense)              (None, 32)                8032      
                                                                 
 Output (Dense)              (None, 1)                 33        
                                                           

## Use a all the pre-trained embedding : Glove/Word2Vec/FastText embedding (this week)

Up to now, the embedding is a matrix of size vocab_size * embedding_size
* vocab_size being the number of tokens in the training data: for example in the previous situation, this size was fixed at 5000
* The objective here is to have a matrix of size pre_traine

In [80]:
# Build the vocabulary list
# Build the embedding matrix
path_to_glove_file = "/content/glove.6B.50d.txt"

vocabulary = []
embedding_matrix = [np.zeros((embedding_dim)),
                    np.zeros((embedding_dim))] # See later : 0=PAD, 1=OOV
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        vocabulary += [word]
        embedding_matrix += [coefs]
embedding_matrix = np.array(embedding_matrix)

In [81]:
embedding_matrix.shape

(400002, 50)

In [82]:
vocab_size = len(embedding_matrix)
len(vocabulary), len(embedding_matrix), 

(400000, 400002)

In [83]:
# Build vectorizer layer and initialize it with the vocabulary list
vectorize_layer = tf.keras.layers.TextVectorization(
        max_tokens=len(embedding_matrix),
        output_mode="int",
        output_sequence_length=max_len,
        vocabulary=vocabulary  # Pass the vocabulary - no need to adapt the layer
                               # Contain the padding token ('') and OOV token ('[UNK]')
)
len(vectorize_layer.get_vocabulary())

400002

In [84]:
# Get the begining of the vocabulary list
vectorize_layer.get_vocabulary()[:10]

['', '[UNK]', 'the', ',', '.', 'of', 'to', 'and', 'in', 'a']

In [85]:
# Test vectorizer layer
vectorize_layer('the sandberger oov')

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([     2, 400001,      1,      0,      0])>

In [86]:
# The rest is similar to an approach with Keras embedding

In [87]:
# Define embedding layer
embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False, # False: don't fine tune the embedding matrix / True: fine tune
    name="Embedding"
)

In [88]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(texts, labels, epochs=10, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(texts, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 80.000001
